In [1]:
def get_secret(secret_name):
    """
    Retrieves a secret value from AWS Secrets Manager.

    This function connects to AWS Secrets Manager in the us-east-1 region and retrieves
    the specified secret. It handles both string and binary secret formats.

    Args:
        secret_name (str): Name/identifier of the secret to retrieve from AWS Secrets Manager

    Returns:
        str: The secret value if stored as a string
        bytes: The decoded secret value if stored as binary

    Raises:
        Exception: If there is an error retrieving the secret from AWS Secrets Manager
    """
    import boto3
    import base64
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(service_name="secretsmanager", region_name=region_name)

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except Exception as e:
        raise e
    else:
        if "SecretString" in get_secret_value_response:
            return get_secret_value_response["SecretString"]
        else:
            return base64.b64decode(get_secret_value_response["SecretBinary"])

In [2]:
def initialize_env():

    import json
    from pathlib import Path
    import os
    import boto3
    
    # RETRIEVING METABASE SECRET
    metabase_secret = json.loads(get_secret("prod/metabase/maxab_config"))
    os.environ["EGYPT_METABASE_USERNAME"] = metabase_secret["metabase_user"]
    os.environ["EGYPT_METABASE_PASSWORD"] = metabase_secret["metabase_password"]
    os.environ["EGYPT_METABASE_URL"] = metabase_secret["metabase_egypt_site"]
    
   
    
    ########### SNOWFLAKE INGESTION credentials ##########
    snowflake_ingestion_secrets = json.loads(get_secret("prod/snowflake/ingestion"))

    os.environ["ingestion_key_bucket"] = snowflake_ingestion_secrets["private_key_bucket"]
    os.environ["ingestion_key_path"] = snowflake_ingestion_secrets["private_key_path"]
    os.environ["ingestion_pass"] = snowflake_ingestion_secrets["encryption_password"]
    os.environ["ingestion_user"] = snowflake_ingestion_secrets["username"]
    os.environ["ingestion_account"] = snowflake_ingestion_secrets["account"]
    os.environ["ingestion_role"] = snowflake_ingestion_secrets["role"]
    


    s3 = boto3.client('s3')

    bucket_name = os.environ["ingestion_key_bucket"]
    key_name = os.environ["ingestion_key_path"]
    s3.download_file(bucket_name, key_name, '/tmp/rsa_key.p8')
    
    # ########## BigQuery credentials ##########
    json_path = str(Path.home())+"/service_account_key.json"
    print(json_path)
    bigquery_key = get_secret("prod/bigquery/sagemaker")
    f = open(json_path, "w")
    f.write(bigquery_key)
    f.close()




In [4]:
def ret_metabase(country, question, use_query=False, filters={}):
   
    from io import StringIO
    import requests
    import pandas as pd
    import json
    import os

    initialize_env()
    
    question_id = str(question)
    
    if country.lower() == 'egypt':
        base_url = 'https://bi.maxab.info/api'
        username = str(os.environ["EGYPT_METABASE_USERNAME"])
        password = str(os.environ["EGYPT_METABASE_PASSWORD"])

    base_headers = {'Content-Type': 'application/json'}


    s_response = requests.post(
        base_url + '/session',
        data=json.dumps({
            'username': username,
            'password': password
        }),
        headers=base_headers)

    s_response.raise_for_status()

    session_token = s_response.json()['id']
    base_headers['X-Metabase-Session'] = session_token

    params = []

    for name, value in filters.items():
        filter_type, filter_value = value
        param = {'target': ['variable', ['template-tag', name]], 'value': filter_value}

        if filter_type.lower() == 'date':
            param['type'] = 'date/range' if isinstance(filter_value, list) else 'date/single'
        elif filter_type.lower() == 'category':
            param['type'] = 'category'
        elif filter_type.lower() == 'text':
            param['type'] = 'text'
        elif filter_type.lower() == 'number':
            param['type'] = 'number'
        elif filter_type.lower() == 'field list':
            param['type'] = 'id'
            param['target'] = ['dimension', ['template-tag', name]]

        params.append(param)

    if use_query:
        p_response = requests.get(base_url + '/card/' + question_id,
                               headers=base_headers)
        rj = p_response.json()
        card = rj.get('dataset_query')
        query = card['native'].get('query','').replace('\n',' ')
        df = snowflake_query(country,query)
        return df

    p_response = requests.post(base_url + '/card/' + question_id + '/query/csv',
                               json={'parameters': params},
                               headers=base_headers)
    p_response.raise_for_status()

    my_dict = p_response.content
    s = str(my_dict, 'utf-8')
    my_dict = StringIO(s)
    df = pd.read_csv(my_dict)
    return(df)

In [7]:
data = ret_metabase("Egypt",61992)
entry = int(data.loc[0,'رقم التليفون' ])

/home/ec2-user/service_account_key.json


In [10]:
entry = int(data.loc[0,'رقم التليفون' ])

In [11]:
entry

1110017085

In [8]:
data

,تاريخ اليوم,كود التاجر,رقم التليفون,سحب,ايداع
0,2025-07-15,83617,1110017085,3356017.27,0.0
1,2025-07-15,731668,1010105276,11110.00,0.0
2,2025-07-15,883214,1200026036,993.00,0.0
3,2025-07-15,475617,1288035635,99.30,0.0
4,2025-07-15,488458,1026181813,4403208.28,0.0
5,2025-07-15,475748,1097603703,5.00,50000.0
6,2025-07-15,880885,1222323498,0.00,3900000.0
7,2025-07-15,880921,1010105297,5560690.99,0.0
8,2025-07-15,880933,1097306909,1966140.00,0.0
9,2025-07-15,880920,1552088812,687647.53,0.0
